# Interactive Map

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.colors as colors

In [ ]:
counties = gpd.read_file('data_files/NI_Counties.shp')
counties.to_crs(epsg=4326, inplace=True)

In [ ]:
xmin, ymin, xmax, ymax = counties.total_bounds
x = xmin+((xmax-xmin)/2)
y = ymin+((ymax-ymin)/2)
centre = [y, x]

In [ ]:
m = folium.Map(centre, zoom_start=8)
m

In [ ]:
sites = gpd.read_file('data_files/Site_Locations.shp')

In [ ]:
sites = sites.rename(columns={'Name': 'Site Name'})

In [ ]:
proximity_info = pd.read_csv('output_files/Proximity_analysis.csv')
characteristics_info = pd.read_csv('output_files/Site_characteristics.csv')

In [ ]:
sites_with_info = sites.merge(proximity_info, on='Site Name')
sites_with_info = sites_with_info.merge(characteristics_info, on='Site Name')

In [ ]:
site_colors = ['orangered', 'olivedrab', 'dodgerblue']

In [ ]:
sites_with_info.explore(m=m, column='Site Name', cmap=colors.ListedColormap(site_colors))
m

In [ ]:
m.save('output_files/Site_Analysis_Results.html') # save to html